In [86]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import interactive
import ipywidgets as widgets
import plotly as ply

In [87]:
class SinusoidTrace(object):
        
    def __init__(self,theta_0,a,omega,n_points=3000,t_start=0,t_stop=3):
        self.t = np.linspace(t_start,t_stop,n_points)
        envelope = np.exp(self.t) 
        self.path = (np.pi/16)*np.sin((2*np.pi*omega)*self.t)*envelope+theta_0
    

def find_reversal_inds(trace):
    pass
#     return reversal_inds
def switch_angle_origin(trace):
    pass
    #Take a trace, and if it's range 0-2

In [88]:
def slider(start,stop,step,init):#,init):
    return widgets.FloatSlider(
    value=init,
    min=start,
    max=stop,
    step=step,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [89]:
def f(theta_0,a,omega):
    
    sinusoidTrace = SinusoidTrace(theta_0,a,omega)
    x = sinusoidTrace.t
    y = sinusoidTrace.path
    fig = plt.figure(figsize=(15,15))
    gs = GridSpec(2,7)
    ax1 = fig.add_subplot(gs[0,0:3])
    plt.plot(x,y%(2*np.pi),'o',markersize=1)
    ax1.set_ylim([0,2*np.pi])
    ax1.set_aspect(1./3)
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)
    plt.xlabel('Time post activation period (min)')
    plt.ylabel('Angular position (radians)')
    ax2 = fig.add_subplot(gs[0,4:6],polar=True)
    plt.plot(y,x)
    plt.grid(False)
    plt.yticks([])

interactive(f, 
            theta_0 = slider(0,2*np.pi,0.1,3*np.pi/2),
            a = slider(0,1,0.05,0.5),
            omega = slider(0,5,0.05,2))
#theta_0: center of oscillation
#a: amplitude of oscillation (scaling factor)
#omega: frequency of oscillation (wrt time)

interactive(children=(FloatSlider(value=4.71238898038469, continuous_update=False, description='theta_0', max=…